In [1]:
from ansatz_simulation_class import AnsatzSimulation
from genetic_quantum import QuanvLayer, QuantumModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from patch_making import PatchExtraction
from torch import tensor
import matplotlib.pyplot as plt

### Instantiating genetic ansatz model

In [2]:
n_qubits = 4
patch_size = 2
num_classes = 2
input_size = 28
toy_chromosome = [['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], [None, 'ctrl_0', 'trgt_0', None], ['rz_gate', 'pauli_y', 'rx_gate', 'phase']]
quanv = QuanvLayer(n_qubits, patch_size, toy_chromosome, mode='2d')
toy_qhcnn = QuantumModel(n_qubits, num_classes, patch_size, input_size, toy_chromosome)
toy_qhcnn

QuantumModel(
  (quanv_layer): QuanvLayer()
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

### Getting data and sampling it

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    PatchExtraction(patch_size)])
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)


In [ ]:
len(mnist_dataset)

In [12]:
selected_classes = [4, 9]
reduced_mnist = [mnist_dataset.__getitem__(index) for index in range(mnist_dataset.__len__()) if mnist_dataset.__getitem__(index)[1] in selected_classes]
len(reduced_mnist)

11791

In [4]:
mnist_dataset.__getitem__(4)

(tensor([[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.9098, 0.9922],
         [0.8235, 0.9882, 0.6588, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.9882],
         [0.7176, 0.0000, 0.3608, 0.9882],
         [0.0824, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.9922, 0.6902, 0.0000, 0.0314],
         [0

In [14]:
class_one = 0
class_two = 0
binary_dataset = []

for image in reduced_mnist:
    if image[1] == 4 and class_one < 50:
        binary_dataset.append(image)
        class_one +=1 
    elif image[1] == 9 and class_two < 50:
        binary_dataset.append(image)
        class_two += 1
        
    if class_one == 50 and class_two == 50:
        break

In [26]:
mnist_images, mnist_labels = zip(*binary_dataset)

In [17]:
tensor_mnist = torch.stack(mnist_images)

In [11]:
tensor_mnist.shape

torch.Size([100, 196, 4])

In [9]:
unfold = nn.Unfold(kernel_size=(patch_size, patch_size), stride=patch_size)
mnist_patches = unfold(tensor_mnist)
n_images, patch_len, n_patches = mnist_patches.shape
mnist_patches = mnist_patches.view((n_images, n_patches, patch_len))

In [10]:
mnist_patches.shape

torch.Size([100, 196, 4])

In [21]:
mnist_patches[0]

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.6235, 0.6235, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8902, 0.9882, 0.9137],
        [0.2235, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.9882, 0.9882],
        [0.9882, 0.3765, 0.9922, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9333],
        [0.9882, 0.0471, 0.4745, 0.0000],
        [0.9922, 0.1961, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.6980, 0.9412, 0.0745, 0.0000],
        [0.0000, 0.0000, 0.9922, 0

In [28]:
mnist_labels = [label%2 for label in mnist_labels]

In [29]:
mnist_labels = tensor(mnist_labels)

In [30]:
mnist_labels

tensor([0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 0])

In [8]:
tensor_mnist.squeeze(1).shape

torch.Size([100, 28, 28])

In [24]:
output = toy_qhcnn.forward(tensor_mnist)

Passing through quanvolution layer...
Image quantum processing time: 9.52734909998253
Quanvolution processing time: 9.61370739992708
Classical layers processing time: 0.0005867998115718365


In [25]:
output

tensor([[-0.6903, -0.6960],
        [-0.7190, -0.6679],
        [-0.6898, -0.6965],
        [-0.7104, -0.6762],
        [-0.6852, -0.7012],
        [-0.7170, -0.6698],
        [-0.7213, -0.6658],
        [-0.7326, -0.6552],
        [-0.7008, -0.6856],
        [-0.7069, -0.6796],
        [-0.7144, -0.6724],
        [-0.7103, -0.6762],
        [-0.7137, -0.6730],
        [-0.7225, -0.6646],
        [-0.7001, -0.6862],
        [-0.6635, -0.7237],
        [-0.6998, -0.6865],
        [-0.6858, -0.7006],
        [-0.6944, -0.6919],
        [-0.6956, -0.6907],
        [-0.7115, -0.6751],
        [-0.7091, -0.6775],
        [-0.6938, -0.6925],
        [-0.7120, -0.6746],
        [-0.7108, -0.6758],
        [-0.6953, -0.6910],
        [-0.7191, -0.6678],
        [-0.7292, -0.6583],
        [-0.6783, -0.7082],
        [-0.6957, -0.6906],
        [-0.6812, -0.7052],
        [-0.7148, -0.6719],
        [-0.7005, -0.6859],
        [-0.7033, -0.6831],
        [-0.7000, -0.6864],
        [-0.6692, -0

### Training my model

In [19]:
from sampled_cv_dataset import SampledDataset4Training

training_zeros_ones = SampledDataset4Training(binary_dataset)

batch_size = 4

training_loader = DataLoader(training_zeros_ones, batch_size, shuffle=True)

In [20]:
training_zeros_ones.__getitem__(0)['image'].shape

torch.Size([196, 4])

In [31]:
optimizer = torch.optim.SGD(toy_qhcnn.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(output, mnist_labels)

In [32]:
loss

tensor(0.6906, grad_fn=<NllLossBackward0>)

In [ ]:
def train_one_epoch(model, epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [33]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

img_grid = make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

TypeError: tensor or list of tensors expected, got <class 'str'>

### Testing tensor dimensions and quantum convolution

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 4
dummy_input = torch.zeros(input_size, input_size, 2).numpy()
patched_images = [image.extract_patches_2d(img, (patch_size, patch_size)) for img in dummy_input]
#dummy_output = quanv.forward(dummy_input)

tensor(patched_images).shape

torch.Size([4, 3, 2, 2])

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 28

shape = (input_size, input_size)
dummy_input = np.zeros(shape, dtype=float)
patched_images = image.extract_patches_2d(dummy_input, (patch_size, patch_size)) 
#dummy_output = quanv.forward(dummy_input)
n_patches, patch_h, patch_w = tensor(patched_images).shape
patched_images = tensor(patched_images).contiguous().view(n_patches, patch_h*patch_w)
image._extract_patches()

In [ ]:
patched_images[0]

tensor([0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
patched_images.shape

torch.Size([729, 4])

In [ ]:
myAnsatz = AnsatzSimulation(n_qubits)
outputs = [myAnsatz.simulate_circuit(patch, 'rx', toy_chromosome) for patch in patched_images]

In [ ]:
tensor(outputs).shape

torch.Size([1458, 4])